In [17]:
import pandas as pd
import numpy as np
import csv

import sqlite3
from itertools import combinations

import bert_score

# Extract data

In [3]:
def extract_data(db_file_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file_path) # './historical_databases/database_trainee/40.db'
    cursor = conn.cursor()

    # Tables
    tables = ['annotation', 'recruitment']

    for table in tables:
        cursor.execute(f"SELECT * FROM {table}")
        data = cursor.fetchall()
        with open(f'output_tables/{table}.csv', 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow([i[0] for i in cursor.description])
            csv_writer.writerows(data)
            
    conn.close()

# Xử lý

In [4]:
extract_data('./historical_databases/database_trainee/40.db')

In [5]:
ann = pd.read_csv('output_tables/annotation.csv')
rcmt = pd.read_csv('output_tables/recruitment.csv')
rcmt = rcmt[['index', 'id']]
rcmt.columns = ['index', 'recruiment_id']

In [6]:
df = pd.merge(rcmt, ann, on='recruiment_id', how='right')
df = df[df.user_id != 1]

In [12]:
counts = df.user_id.value_counts().sort_index()
indexes = [2, 3, 4, 6, 7, 8, 9, 10, 11]
names = ['VQuoc', 'TDuong', 'BKhanh', 'QNhu', 'TDinh', 'HGiang', 'BHan', 'Kiet', 'HAnh']
index_to_name = dict(zip(indexes, names))
index_to_count = counts.to_dict()

df_counts = pd.DataFrame({'name': index_to_name, 'count': index_to_count})
df_counts = df_counts.set_index(df_counts.index)
df_counts['count'].fillna(0, inplace=True)
print(df_counts.T)

          2       3       4     6      7       8     9     10    11
name   VQuoc  TDuong  BKhanh  QNhu  TDinh  HGiang  BHan  Kiet  HAnh
count    250     250     250   249    249     249   250   249   249


In [13]:
def fill_annotator(annos):
    index_to_name_reverse = {v: k for k, v in index_to_name.items()}
    re = [index_to_name_reverse[name] for name in annos]
    return df[df.user_id.map(lambda x: x in re)]

# df = fill_annotator(['VQuoc', 'TDuong', 'BKhanh', 'QNhu', 'TDinh', 'HGiang', 'BHan', 'Kiet', 'HAnh'])
df = fill_annotator(['VQuoc', 'TDuong', 'BKhanh'])

## 3. BERT Score

In [20]:
pv_table_expl = df.pivot(index='recruiment_id', columns='user_id', values='explanation')
pv_table_expl.columns = pv_table_expl.columns.map(index_to_name)
pv_table_expl.dropna(inplace=True)
pv_table_expl.head()

user_id,VQuoc,TDuong,BKhanh
recruiment_id,,,
34380155,Không nhất quán về hình thức trả lương và mức ...,"Thông tin rõ ràng, minh bạch","Mức lương đề cập chưa có sự nhất quán, địa chỉ..."
43954033,"Thông tin công việc rõ ràng, tuy nhiên quyền l...",Thông tin lương chưa đề cập,Chưa đề cập đến mức lương công việc.
59175011,"Mức lương trả không nhất quán, yêu cầu CCCD gốc.","Thông tin tiêu đề chứa tiền lương, nhưng không...","Mức lương trả không nhất quán, yêu cầu CCCD gốc."
61072020,Tiêu đề công việc không phù hợp và chưa đề cập...,"Tiêu đề công việc hấp dẫn ,không rõ ràng công ...",Tiêu đề tin tuyển dụng chứa các từ hấp dẫn ngư...
62505030,"Thông tin công việc không nhất quán, yêu cầu C...","Thông tin rõ ràng, minh bạch",Chưa đề cập rõ ràng công ty tuyển dụng.


In [21]:
def bert_score_each_sample(li_expl_sents):
    # li_expl_sents = sent_1, sent_2, sent_3,...
    precisions, recalls, f1s = [], [], []
    combinations_two = list(combinations(li_expl_sents, 2))
    for comb in combinations_two:
        precision, recall, f1 = bert_score.score(
            [comb[0]], [comb[1]],
            model_type='bert-base-multilingual-cased',
            lang='vi', 
            verbose=True
        )
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)

    return np.mean(precisions), np.mean(recalls), np.mean(f1s)

def bert_score_func(df, x, y):
    df = df.to_numpy()[x:y]
    precisions, recalls, f1s = [], [], []
    for i, sample in enumerate(df):
        precision, recall, f1 = bert_score_each_sample(sample)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    
    print(f"################### {x} - {y} ###################")
    print('R1: ', np.mean(precisions))
    print('R2: ', np.mean(recalls))
    print('R3: ', np.mean(f1s))

In [23]:
bert_score_func(pv_table_expl, 0, 3)

KeyboardInterrupt: 